In [1]:
# Clear existing environment variables
import os
os.environ.pop('TOGETHER_API_KEY', None)  # Remove existing key

# Force reload from .env file
from dotenv import load_dotenv
load_dotenv(override=True)  # override=True forces reload

# Reinitialize Together client
import together
together.api_key = os.getenv('TOGETHER_API_KEY')

# Verify the new key is loaded
print(f"New API key loaded: {'*' * (len(os.getenv('TOGETHER_API_KEY', '')) - 4) + (os.getenv('TOGETHER_API_KEY', '')[-4:] if os.getenv('TOGETHER_API_KEY') else 'None')}")

New API key loaded: **********************************************sYdw


In [2]:
# import model_util helper functions
from model_utils import init_wandb, track_experiment

In [3]:
# Login to wandb
import wandb
wandb.login() 
init_wandb()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arsalon-ai (arsalon-amini-ai). Use `wandb login --relogin` to force relogin


Initialized wandb run: experiment_20250306_022501


True

In [4]:
# Simple Zero Shot Prompt Experiment

# define the prompt
prompt = "Write a haiku about AI in 13 words."
response1 = track_experiment(prompt, experiment_type="basic-haiku", run_id="1")
response2 = track_experiment(prompt, experiment_type="basic-haiku", run_id="2")
response3 = track_experiment(prompt, experiment_type="basic-haiku", run_id="3")

print(response1)

Artificial minds rising, intelligent systems learning, growing every single day slowly


In [5]:
from pdf_util import extract_text_from_pdf, count_tokens, extract_sections

MAX_TOKENS = 10000  # Llama-3.3-70B-Instruct context window size
RESERVE_TOKENS = 500  # Reserve some tokens for the response

extracted_text = extract_text_from_pdf('literature/human_enhancement_genetic_engineering.pdf')

token_count = count_tokens(extracted_text)
print(f"Number of tokens in the PDF: {token_count}")

sections = extract_sections(extracted_text)
print(f"Found {len(sections)} sections.")

for section_title, section_content in sections.items():
    token_count = count_tokens(section_content)
    print(f"\nProcessing section '{section_title}' with {token_count} tokens")
    
    # # If this section is too long, truncate it to make sure the prompt fits within the safe token limit.
    # if token_count > (MAX_TOKENS - RESERVE_TOKENS):
    #     print(f"Warning: Section '{section_title}' exceeds the token limit. Truncating content.")
    #     section_content = section_content[:(MAX_TOKENS - RESERVE_TOKENS)]
    
    # # Build the prompt to include the section title and content
    # prompt = (
    #     f"Section Title: {section_title}\n\n"
    #     f"Section Text: {section_content}\n\n"
    #     "Please provide a concise summary for the above section in exactly 5 words."
    # )
    
    # # Call the language model function to get the summary of the section
    # summary = llama(prompt)
    # print(f"Summary for section '{section_title}': {summary}")

# if token_count > (MAX_TOKENS - RESERVE_TOKENS):
#     print(f"Warning: Input text is too long ({token_count} tokens).")
#     print(f"Maximum safe input length is {MAX_TOKENS - RESERVE_TOKENS} tokens (reserving {RESERVE_TOKENS} tokens for response)")
#     # Optionally, you could truncate the text here
# else:
#     print(f"Text length is within limits: {token_count} tokens")
#     prompt = f"given {extracted_text}, write a summary of the document in 5 words"
#     response = llama(prompt)
#     print(response)



Number of tokens in the PDF: 8430
Found 3 sections.

Processing section 'Abstract' with 273 tokens

Processing section 'Introduction' with 6901 tokens

Processing section 'Methods' with 998 tokens


In [6]:
from pdf_util import count_tokens

# Extract just Abstract and Methods sections and combine them
abstract_text = sections.get('Abstract', '')
methods_text = sections.get('Methods', '')
combined_text = f"Abstract:\n{abstract_text}\n\nMethods:\n{methods_text}"

token_count = count_tokens(combined_text)
print(f"Number of tokens in the PDF: {token_count}")

Number of tokens in the PDF: 1276


In [8]:
# Few Shot Prompt Experiment

prompt = (
    f"Below are a few example evaluations of research papers in the field of genetic engineering:\n\n"
    
    f"Example 1:\n"
    f"Combined Abstract and Methods:\n"
    f"\"Abstract: Recent advances in CRISPR/Cas9 have enabled precise modifications in crop genomes to enhance drought tolerance. "
    f"Methods: The study used CRISPR vectors to introduce targeted mutations in drought-resistance genes of maize.\"\n"
    f"Evaluation: The research examines how targeted gene editing can improve crop resilience to environmental stress.\n"
    f"Novel hypothesis: Optimize CRISPR protocols for enhanced drought resilience in maize.\n\n"
    
    f"Example 2:\n"
    f"Combined Abstract and Methods:\n"
    f"\"Abstract: A novel gene therapy approach was explored to combat muscular dystrophy by correcting mutant genes. "
    f"Methods: Viral vectors were employed to deliver corrected gene sequences to the affected muscle tissues.\"\n"
    f"Evaluation: The study tests the feasibility of gene therapy in alleviating symptoms of genetic muscle disorders.\n"
    f"Novel hypothesis: Enhance the specificity of viral vector delivery in gene therapy for muscular dystrophy.\n\n"
    
    f"Now, here is the combined Abstract and Methods section of a research paper:\n\n"
    f"{combined_text}\n\n"
    "Evaluate the research hypothesis and methods and generate a novel hypothesis for future experimentation."
)

fewshot_response = track_experiment(prompt, experiment_type="fewshot-prompt")
print(fewshot_response)

wandb:   1 of 1 files downloaded.  


Evaluation: 
The research commentary explores the implications of genetic engineering on human biology, evolution, and the natural environment. It highlights the importance of considering multiple effects (pleiotropy) and complex epigenetic interactions among genotype, phenotype, and ecology. The authors also propose the need for a practicable distinction between 'therapy' and 'enhancement' in future regulations. The commentary is based on a comprehensive review of existing literature, including studies on genome editing, evolutionary biology, and the ethics of genetic engineering.

The methods used in this research are primarily based on a review of existing literature, including scientific articles, books, and reports from reputable sources. The authors have compiled a list of references that provide a foundation for their discussion on genetic engineering, evolution, and ethics. However, the commentary does not present any new experimental data or primary research.

Novel Hypothesis

In [6]:
# Role Prompt Experiment
role_prompt = (
    "You are an expert research scientist specializing in genetic engineering with extensive experience "
    "in analyzing experimental designs and research methodologies. "
    "Your task is twofold:\n\n"
    
    "1. Evaluate the following combined Abstract and Methods section of a research paper by highlighting the key strengths and weaknesses in the existing hypothesis and experimental design.\n\n"
    
    "2. Based on your evaluation, propose a novel hypothesis for future experimentation that addresses any gaps or builds upon the findings.\n\n"
    
    "Combined Abstract and Methods section:\n"
    f"{combined_text}\n\n"
    
    "Please provide a concise evaluation followed by your novel hypothesis."
)

role_response = llama(role_prompt)
print(role_response)

**Evaluation of the Combined Abstract and Methods Section:**

The paper provides a comprehensive commentary on genetic engineering in an evolutionary context, highlighting the importance of considering multiple effects (pleiotropy) and complex epigenetic interactions. The authors emphasize the need to distinguish between 'therapy' and 'enhancement' and propose that this distinction should be effectively implemented in future regulations.

Key strengths:

1. Interdisciplinary approach: The paper combines insights from evolutionary biology, developmental biology, and other disciplines to provide a comprehensive understanding of genetic engineering.
2. Emphasis on pleiotropy and epigenetic interactions: The authors highlight the importance of considering the complex interactions between genotype, phenotype, and ecology.

Key weaknesses:

1. Lack of empirical evidence: The paper is primarily a commentary, and the authors do not provide new empirical evidence to support their claims.
2. Lim

In [7]:
# Chain of Thought Prompt Experiment
cot_prompt = (
    "You are an expert research scientist specializing in genetic engineering with extensive experience in evaluating research papers. "
    "When analyzing the following combined Abstract and Methods section, list your thoughts, using your internal chain-of-thought, to consider the strengths and weaknesses of the hypothesis and experimental design, as well as any gaps or opportunities for future research. "
    "Then, provide only your final, concise evaluation along with a novel hypothesis for future experimentation. "
    "Combined Abstract and Methods section:\n\n"
    f"{combined_text}\n\n"
    "Please provide your final evaluation and novel hypothesis."
)

cot_response = llama(cot_prompt)
print(cot_response)


**Final Evaluation:** The provided abstract and methods section present a comprehensive commentary on genetic engineering in an evolutionary context, highlighting the importance of considering pleiotropy, epigenetic interactions, and the unit of impact on human populations and the environment. The authors emphasize the need for a distinction between 'therapy' and 'enhancement' and stress the importance of interdisciplinary discussions. However, the text lacks a clear hypothesis and experimental design, instead focusing on reviewing existing literature and proposing a framework for future discussions.

**Novel Hypothesis:** "The introduction of genetically engineered traits in human populations will lead to unintended evolutionary consequences, such as the disruption of existing gene regulatory networks, which can be mitigated by developing a systems biology approach that integrates evolutionary, developmental, and environmental factors to predict and monitor the long-term effects of ge

In [10]:
# Evaluator LLM for Prompt Engineering Experiment

evaluator_prompt = (
    "You are a scientific evaluator with expertise in genetic engineering. "
    "Your task is to assess the research evaluation outputs produced by three different prompting approaches: Few Shot, Role Based, and Chain of Thought. "
    "Each of these outputs includes an assessment of the research paper's hypothesis and a proposal for a novel hypothesis for future experimentation.\n\n"
    
    "Please evaluate each of the provided outputs based on the following criteria:\n"
    "1. Novelty and ingenuity: How original and innovative is the proposed hypothesis given the combined Abstract and Methods section?\n"
    "2. Experimental design feasibility: How feasible is the experimental design suggested for testing the hypothesis?\n"
    "3. Impact potential: What is the potential impact of this research on the field of genetic engineering?\n"
    "4. Cost and materials feasibility: Evaluate whether the required materials and overall cost make the experimental design practical.\n\n"
    
    "Below is the Combined Abstract and Methods section for context:\n\n"
    f"{combined_text}\n\n"
    
    "Now, please review and evaluate the following outputs:\n\n"
    
    "----- Few Shot Output -----\n"
    f"{fewshot_response}\n\n"
    
    "----- Role Based Output -----\n"
    f"{role_response}\n\n"
    
    "----- Chain of Thought Output -----\n"
    f"{cot_response}\n\n"
    
    "For each output, provide a detailed evaluator report that includes numerical ratings on a scale of 1 to 5 along with concise justifications for each of the four criteria. "
    "Your response should clearly indicate your evaluation for each of the three outputs."
)

evaluation_report = llama(evaluator_prompt)
print(evaluation_report)

**Evaluator Report**

### Few Shot Output

1. **Novelty and ingenuity**: 4
The proposed hypothesis investigates the potential long-term consequences of germline editing on human evolution, which is a novel and intriguing direction. The focus on the interplay between genetic modifications, environmental pressures, and epigenetic factors adds complexity and depth to the hypothesis. However, the idea of exploring the evolution of complex traits is not entirely new, which prevents a perfect score.

2. **Experimental design feasibility**: 4
The suggested experimental design involves a combination of computational modeling, simulation studies, and empirical research, which is feasible and well-rounded. The incorporation of data from various fields, including genetics, evolutionary biology, ecology, and anthropology, adds strength to the design. However, the development of new methodologies for predicting and assessing the long-term effects of genetic engineering might pose some challenges.

